In [55]:
import warnings
warnings.filterwarnings('ignore')

In [56]:
from crewai import Agent,Task,Crew,LLM

In [57]:
from dotenv import  load_dotenv
import os
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
serper_api_key=os.getenv("SERPER_API_KEY")

os.environ["OPENAI_API_KEY"] = "dummy"

llm = LLM(model="gemini/gemini-2.5-flash",
          provider="gemini",
          temperature=0.7,
          api_key=api_key)

In [58]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [59]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=llm
)

In [60]:
# Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=llm
)

In [61]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=llm
)

In [62]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [63]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [64]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    agent=logistics_manager
)

In [65]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [66]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    manager_llm=llm,
    verbose=True,

)

In [67]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [68]:
result = event_management_crew.kickoff(inputs=event_details)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 89a2737a-6d30-4158-95ed-ae76b738b16e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Venue Coordinator                                                                                       │
│                                                                                                                 │
│  Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Venue Coordinator                                                                                       │
│                                                                                                                 │
│  Thought: Action: Search the internet with Serper                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"venues in San Francisco for tech conference\"}"                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'venues in San Francisco for tech conference', 'type': 'search', 'num': 10,         │
│  'engine': 'google'}, 'organic': [{'title': 'Conference Venues for Rent in San Francisco, CA', 'link':          │
│  'https://www.tagvenue.com/us/hire/conference-venues/san-francisco', 'snippet': 'Tagvenue offers an extensive   │
│  range of conference venues in San Francisco, making it easy for you to find the perfect one. Select and book   │
│  your venue today!', 'position': 1}, {'title': 'Rent a networking venue in San Francisco, CA', 'link':          │
│  'https://www.peerspace.com/venues/san-francisco--ca/networking-venue', 'snippet': 'The Nook. A community       │
│  event space located on Market Street, right next to BART and Muni lines. Our brick walls, full sound and       │
│  lighting systems, and wooden ...', 'position': 2}, {'title': 'The Best 12 Tech Conferences to Attend in San    │
│  Francisco ...', 'link': 'https://vanguard-x.com/tech-events-2025/tech-conferences-san-francisco-2025/',        │
│  'snippet': "San Francisco's tech scene heats up with 12 must-attend conferences from July to December 2025     │
│  for startups and investors.", 'position': 3}, {'title': 'Top 20 Corporate Event Venues in San Francisco, CA',  │
│  'link': 'https://www.partyslate.com/find-venues/corporate-event-venues/near/san-francisco-ca-usa', 'snippet':  │
│  'Pier 27 offers almost 55,000 sq. ft. of flexible event space. Whether it is for a conference, a gala, a       │
│  product launch or a wedding, Pier 27 is a great option ...', 'position': 4}, {'title': 'Best event venues for  │
│  a tech meetup : r/bayarea', 'link':                                                                            │
│  'https://www.reddit.com/r/bayarea/comments/1dt8ok5/best_event_venues_for_a_tech_meetup/', 'snippet': 'Hey      │
│  all, looking for suggestions including breweries/coffee shop/event space holding upto 30-40 ppl in the Bay     │
│  Area from San Mateo to Santa ...', 'position': 5}, {'title': '8 Meeting Venues Within One Block of the         │
│  Moscone Center', 'link':                                                                                       │
│  'https://www.sftravel.com/meetings/article/meeting-venues-within-one-block-moscone-center', 'snippet': 'There  │
│  are 8 unique meeting ...                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Logistics Manager                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Convene 100 Stockton offers comprehensive in-house logistics arrangements including catering with              │
│  customization options and in-room A/V technology supported by an on-site team. Dedicated hospitality           │
│  resources are also provided. The venue's largest single event space, Pacific Hall, has a maximum capacity of   │
│  468 people. The venue can accommodate events up to just under 470 participants in total. For an event with     │
│  500 participants, this venue may require utilizing multiple spaces or further discussion with the venue        │
│  directly to confirm feasibility and arrangements for the full attendee count on 2024-09-15. Specific catering  │
│  menus, pricing, and detailed equipment lists would be coordinated through Convene 100 Stockton's inquiry       │
│  process.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 ## Final Result: Convene 100 Stockton offers comprehensive in-house logistics arrangements including catering with customization options and in-room A/V technology supported by an on-site team. Dedicated hospitality resources are also provided. The venue's largest single event space, Pacific Hall, has a maximum capacity of 468 people. The venue can accommodate events up to just under 470 participants in total. For an event with 500 participants, this venue may require utilizing multiple spaces or further discussion with the venue directly to confirm feasibility and arrangements for the full attendee count on 2024-09-15. Specific catering menus, pricing, and detailed equipment lists would be coordinated through Convene 100 Stockton's inquiry process.
 

=====
## HUMAN FEEDBACK: Provide feedback on the Final Result and Agent's actions.
Please follow these guidelines:
 - If you are happy with the result, simply hit Enter without typing anything.
 - Otherwise, provide specific improvement

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4e8635e9-e37f-4843-8c1c-da421f91dd1a                                                                     │
│  Agent: Logistics Manager                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[EventBus Error] Handler 'on_agent_logs_started' failed for event 'AgentLogsStartedEvent': <ContextVar name='parent_header' at 0x108474310>
[EventBus Error] Handler 'on_agent_logs_execution' failed for event 'AgentLogsExecutionEvent': <ContextVar name='parent_header' at 0x108474310>
[EventBus Error] Handler 'on_agent_logs_execution' failed for event 'AgentLogsExecutionEvent': <ContextVar name='parent_header' at 0x108474310>
[EventBus Error] Handler 'on_agent_logs_execution' failed for event 'AgentLogsExecutionEvent': <ContextVar name='parent_header' at 0x108474310>
[EventBus Error] Handler 'on_task_completed' failed for event 'TaskCompletedEvent': <ContextVar name='parent_header' at 0x108474310>


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 89a2737a-6d30-4158-95ed-ae76b738b16e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Tech Innovation Conference: Marketing Activities and Attendee Engagement Report                │
│                                                                                                                 │
│  ## Goal: Engage at least 500 potential attendees for the Tech Innovation Conference.                           │
│                                                                                                                 │
│  This report outlines the proposed marketing activities designed to maximize event exposure and participation,  │
│  along with strategies for fostering and measuring attendee engagement.                                         │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Marketing Activities                                                                                       │
│                                                                                                                 │
│  Our marketing strategy will leverage a multi-channel approach, focusing on digital platforms to reach a broad  │
│  and relevant audience.                                                                                         │
│                                                                                                                 │
│  1.  **Dedicated Event Website/Landing Page:**                                                                  │
│      *   **Activity:** Create a professional, mobile-responsive website serving as the central hub for all      │
│  conference information. This includes speaker bios, agenda, session details, registration links, venue         │
│  information, and sponsorship opportunities. The site will feature compelling visuals and clear calls to        │
│  action.                                                                                                        │
│      *   **Engagement Driver:** Provides a single source of truth, enabling easy access to information and a    │
│  seamless registration process. Interactive elements like a countdown timer and featured speaker spotlights     │
│  will build anticipation.                                                                                       │
│                                                                                                                 │
│  2.  **Content Marketing:**                                                                                     │
│      *   **Activity:** Develop a content calendar featuring blog posts, articles, and short videos related to   │
│  the conference themes (e.g., "Top 5 Tech Innovations to Watch," "Why AI is Reshaping Industries"). These will  │
│  be published on the conference website and shared across social media. Testimonials from previous attendees    │
│  or sneak peeks of speaker topics will also be utilized.                                                        │
│      *   **Engagement Driver:** Attracts potential att

In [69]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': "40 O'Farrell Street, San Francisco, CA 94108",
 'booking_status': 'Available for inquiry/booking',
 'capacity': 1900,
 'name': 'Convene 100 Stockton'}


In [70]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Tech Innovation Conference: Marketing Activities and Attendee Engagement Report

## Goal: Engage at least 500 potential attendees for the Tech Innovation Conference.

This report outlines the proposed marketing activities designed to maximize event exposure and participation, along with strategies for fostering and measuring attendee engagement.

---

### Marketing Activities

Our marketing strategy will leverage a multi-channel approach, focusing on digital platforms to reach a broad and relevant audience.

1.  **Dedicated Event Website/Landing Page:**
    *   **Activity:** Create a professional, mobile-responsive website serving as the central hub for all conference information. This includes speaker bios, agenda, session details, registration links, venue information, and sponsorship opportunities. The site will feature compelling visuals and clear calls to action.
    *   **Engagement Driver:** Provides a single source of truth, enabling easy access to information and a seamless registration process. Interactive elements like a countdown timer and featured speaker spotlights will build anticipation.

2.  **Content Marketing:**
    *   **Activity:** Develop a content calendar featuring blog posts, articles, and short videos related to the conference themes (e.g., "Top 5 Tech Innovations to Watch," "Why AI is Reshaping Industries"). These will be published on the conference website and shared across social media. Testimonials from previous attendees or sneak peeks of speaker topics will also be utilized.
    *   **Engagement Driver:** Attracts potential attendees by providing valuable insights and demonstrating the relevance and quality of the conference content. Encourages sharing and discussion around key topics.

3.  **Email Marketing Campaigns:**
    *   **Activity:** Implement a segmented email marketing strategy targeting different groups (e.g., past attendees, industry professionals, potential sponsors). This will include early bird announcements, speaker spotlights, agenda highlights, last-chance registration reminders, and practical information for registered attendees.
    *   **Engagement Driver:** Nurtures leads and keeps interested individuals informed and excited. Personalized content can increase open rates and click-throughs, driving registrations and pre-event engagement.

4.  **Social Media Marketing:**
    *   **Activity:** Establish a strong presence on platforms like LinkedIn, Twitter, and Facebook. Share conference updates, speaker announcements, behind-the-scenes content, and engaging questions related to tech trends. Utilize relevant hashtags (`#TechInnovationConf`, `#Innovation2024`) and encourage user-generated content. Run targeted ad campaigns on these platforms.
    *   **Engagement Driver:** Increases brand visibility, fosters community, and facilitates direct interaction with potential attendees. Live Q&A sessions with speakers or polls can significantly boost engagement.

5.  **Influencer and Speaker Promotion:**
    *   **Activity:** Encourage confirmed speakers and industry influencers to promote their involvement in the conference to their networks. Provide them with ready-to-use social media kits and unique registration links for tracking.
    *   **Engagement Driver:** Leverages the credibility and reach of key figures, expanding the conference's audience and attracting attendees who value the insights of these individuals.

6.  **Partnerships and Cross-Promotion:**
    *   **Activity:** Collaborate with relevant industry associations, media outlets, and tech companies to cross-promote the conference. This could involve newsletter mentions, banner ads on partner websites, or co-hosted webinars.
    *   **Engagement Driver:** Taps into established communities and expands reach to highly targeted audiences who are already interested in tech innovation.

7.  **Early Bird and Group Discounts:**
    *   **Activity:** Offer tiered pricing with significant discounts for early registrations and groups.
    *   **Engagement Driver:** Incentivizes early commitment and encourages teams to attend together, boosting overall registration numbers.

---

### Attendee Engagement and Measurement

Attendee engagement will be a continuous focus, from pre-event excitement to post-event follow-up. We will track both quantitative and qualitative metrics.

**Pre-Event Engagement:**

*   **Website Traffic & Conversions:**
    *   **Activities:** Monitor website visits, page views (especially agenda and speaker pages), time spent on site, and bounce rate using analytics tools. Track registration form completions.
    *   **Measurement:** Google Analytics will provide data on traffic sources, user behavior, and conversion rates. High engagement here indicates strong initial interest.
*   **Email Campaign Performance:**
    *   **Activities:** Track open rates, click-through rates (CTR) on registration links, and unsubscribe rates for all email communications.
    *   **Measurement:** Email marketing platform (e.g., Mailchimp, Constant Contact) analytics will provide detailed metrics on campaign effectiveness. High CTR on registration links directly correlates with attendee acquisition.
*   **Social Media Reach & Interaction:**
    *   **Activities:** Monitor follower growth, impression counts, likes, shares, comments, and mentions of conference hashtags. Track engagement with promoted posts and ads.
    *   **Measurement:** Native analytics on LinkedIn, Twitter, and Facebook, along with social listening tools, will provide insights into audience interest and virality.
*   **Pre-Conference Webinars/Q&A Sessions:**
    *   **Activities:** Host free online sessions with key speakers or organizers to discuss hot topics or answer questions about the conference.
    *   **Measurement:** Track attendance numbers, live poll participation, and question submission rates.

**During-Event Engagement (Assumed Hybrid/In-Person with Digital Tools):**

*   **Event App Usage:**
    *   **Activities:** If an event app is used, track session attendance (virtual and in-person check-ins), personalized agenda creation, networking interactions (e.g., meeting requests, profile views), poll participation, and Q&A submissions.
    *   **Measurement:** Event app analytics will provide granular data on attendee behavior and interaction with content and other attendees.
*   **Session Participation:**
    *   **Activities:** Monitor attendance at individual sessions, active participation in live Q&A, and engagement with interactive elements (e.g., live polls, surveys).
    *   **Measurement:** Session attendance tracking (via app or scanning), number of questions asked, and poll response rates.
*   **Social Media Buzz:**
    *   **Activities:** Continue monitoring conference hashtags for attendee posts, photos, and discussions.
    *   **Measurement:** Real-time social listening tools to gauge the sentiment and volume of discussions.
*   **Networking Activities:**
    *   **Activities:** Observe participation in dedicated networking events, speed networking sessions, or virtual meeting rooms.
    *   **Measurement:** Number of connections made via the event app, attendance at networking events.
*   **Exhibitor/Sponsor Interaction:**
    *   **Activities:** Track visits to exhibitor booths (physical or virtual), content downloads, and lead scans.
    *   **Measurement:** Data provided by exhibitors/sponsors on interactions.

**Post-Event Engagement:**

*   **Feedback Surveys:**
    *   **Activities:** Distribute comprehensive post-conference surveys to all attendees, gathering feedback on content quality, speaker effectiveness, networking opportunities, overall satisfaction, and likelihood to recommend.
    *   **Measurement:** Analyze survey response rates, satisfaction scores, and Net Promoter Score (NPS).
*   **Content Consumption:**
    *   **Activities:** Make session recordings and presentation slides available post-event. Track views and downloads of this content.
    *   **Measurement:** Video platform analytics (e.g., YouTube, Vimeo) and website download metrics.
*   **Community Growth:**
    *   **Activities:** Monitor growth in post-conference online communities (e.g., LinkedIn groups) and ongoing discussions.
    *   **Measurement:** Membership numbers and activity levels in dedicated groups.
*   **Re-registration for Future Events:**
    *   **Activities:** Offer early bird registration for the next conference.
    *   **Measurement:** Track the percentage of current attendees who register for the subsequent event.

---

By implementing these marketing activities and diligently tracking the outlined engagement metrics, we aim to not only meet our target of 500 potential attendees but also cultivate a highly engaged and satisfied audience for the Tech Innovation Conference.